In [18]:
import os
import logging

from pyflink.common import WatermarkStrategy
from pyflink.datastream import StreamExecutionEnvironment, RuntimeExecutionMode
from pyflink.datastream.connectors.kafka import KafkaSource, KafkaOffsetsInitializer
from pyflink.datastream.formats.json import JsonRowDeserializationSchema
from pyflink.common import Types, Row
from pyflink.datastream.connectors.jdbc import JdbcSink, JdbcConnectionOptions, JdbcExecutionOptions
from pyflink.datastream.connectors.kafka import KafkaTopicPartition
import json
from datetime import datetime

RUNTIME_ENV = os.getenv("RUNTIME_ENV", "local")
BOOTSTRAP_SERVERS = os.getenv("BOOTSTRAP_SERVERS", "localhost:9092")

In [19]:
env = StreamExecutionEnvironment.get_execution_environment()
env.set_runtime_mode(RuntimeExecutionMode.STREAMING)

JavaObject id=o365

In [20]:
jar_files = ["flink-sql-connector-kafka-3.2.0-1.18.jar", "postgresql-42.6.0.jar", "flink-connector-jdbc-3.1.2-1.18.jar"]

In [21]:
CURRENT_DIR = os.getcwd()

In [22]:
departure_type = Types.ROW_NAMED(['delay', 'time', 'uncertainty'], [Types.INT(), Types.INT(), Types.INT()])
arrival_type = Types.ROW_NAMED(['delay', 'time', 'uncertainty'], [Types.INT(), Types.INT(), Types.INT()])
stop_time_update_type = Types.ROW_NAMED(['stop_sequence', 'arrival', 'departure', 'stop_id', 'schedule_relationship'], [Types.INT(), arrival_type, departure_type, Types.STRING(), Types.INT()])
vehicle_type_info = Types.ROW_NAMED(['id', 'label', 'license_plate'], [Types.STRING(), Types.STRING(), Types.STRING()])
trip_update_trip_info = Types.ROW_NAMED(['trip_id', 'start_time', 'start_date', 'schedule_relationship', 'route_id', 'direction_id'], [Types.STRING(), Types.STRING(), Types.STRING(), Types.INT(), Types.STRING(), Types.INT()])
trip_update_type = Types.ROW_NAMED(['trip', 'stop_time_update', 'vehicle', 'timestamp', 'delay'],[trip_update_trip_info, Types.OBJECT_ARRAY(stop_time_update_type), Types.STRING(), Types.STRING(), Types.INT()])
trip_info_type = Types.ROW_NAMED(['id', 'is_deleted', 'trip_update', 'vehicle', 'alert'], [Types.STRING(), Types.BOOLEAN(), trip_update_type, Types.STRING(), Types.STRING()])
json_format = JsonRowDeserializationSchema.builder().type_info(trip_info_type).build()

In [23]:
jar_paths = tuple(
            [f"file://{os.path.join(CURRENT_DIR, 'Downloads', name)}" for name in jar_files]
        )
logging.info(f"adding local jars - {', '.join(jar_files)}")
env.add_jars(*jar_paths)

In [24]:
partition_set = {
    KafkaTopicPartition("transitStream", 0)
}

flink_simple_json_source = (
        KafkaSource.builder()
        .set_bootstrap_servers(BOOTSTRAP_SERVERS)
        .set_group_id("flink.testertransit")
        .set_starting_offsets(KafkaOffsetsInitializer.earliest())
        .set_value_only_deserializer(
            json_format
        )
        .set_partitions(partition_set)
        .build()
    )

In [25]:
flink_stream = env.from_source(
        flink_simple_json_source, WatermarkStrategy.no_watermarks(), "what is this"
    )

In [26]:
def extract_trip_info(row):
    trip_id = row.id
    start_time = datetime.strptime(row.trip_update.trip.start_time, "%H:%M:%S")
    start_date = datetime.strptime(row.trip_update.trip.start_date, "%Y%m%d")
    for stops in row.trip_update.stop_time_update:
        stop_seq = stops.stop_sequence
        if stops.departure is not None:
            if stops.departure.delay > 0:
                delay_in_seconds = stops.departure.delay
                status = "delayed"
            elif stops.departure.delay < 0:
                delay_in_seconds = abs(stops.departure.delay)
                status = "early"
            else:
                delay_in_seconds = stops.departure.delay
                status = "running on time"
            stop_time = datetime.fromtimestamp(stops.departure.time)
        else:
            if stops.arrival.delay > 0:
                delay_in_seconds = stops.arrival.delay
                status = "delayed"
            elif stops.arrival.delay < 0:
                delay_in_seconds = abs(stops.arrival.delay)
                status = "early"
            else:
                delay_in_seconds = stops.arrival.delay
                status = "running on time"
            stop_time = datetime.fromtimestamp(stops.arrival.time)
        yield Row(trip_id, start_time, start_date, stop_seq, stop_time, delay_in_seconds, status)

In [27]:
row_type_info = Types.ROW([Types.STRING(), Types.SQL_TIME(), Types.SQL_DATE(), Types.INT(),Types.SQL_TIMESTAMP(), Types.INT(), Types.STRING()])

flink_stream.flat_map(extract_trip_info, output_type = row_type_info).add_sink(
    JdbcSink.sink(
        """INSERT INTO public.trip_updates (trip_id, start_time, start_date, stop_seq, stop_time, delay_in_sec, status)
            VALUES (?, ?, ?, ?, ?, ?, ?) 
            ON CONFLICT (trip_id, start_date, stop_seq, u) 
            DO UPDATE SET 
            trip_id = EXCLUDED.trip_id,
            start_time = EXCLUDED.start_time,
            start_date = EXCLUDED.start_date,
            stop_seq = EXCLUDED.stop_seq,
            stop_time = EXCLUDED.stop_time,
            delay_in_sec = EXCLUDED.delay_in_sec,
            status = EXCLUDED.status
        """,
        row_type_info,
        JdbcConnectionOptions.JdbcConnectionOptionsBuilder()
            .with_url('jdbc:postgresql://localhost:5432/transitstreamtest')
            .with_driver_name('org.postgresql.Driver')
            .with_user_name('root')
            .with_password('root')
            .build(),
        JdbcExecutionOptions.builder()
            .with_batch_interval_ms(1000)
            .with_batch_size(200)
            .with_max_retries(2)
            .build()
)
)


In [28]:
env.execute("")

Py4JJavaError: An error occurred while calling o354.execute.
: org.apache.flink.runtime.client.JobExecutionException: Job execution failed.
	at org.apache.flink.runtime.jobmaster.JobResult.toJobExecutionResult(JobResult.java:144)
	at org.apache.flink.runtime.minicluster.MiniClusterJobClient.lambda$getJobExecutionResult$3(MiniClusterJobClient.java:141)
	at java.base/java.util.concurrent.CompletableFuture$UniApply.tryFire(CompletableFuture.java:646)
	at java.base/java.util.concurrent.CompletableFuture.postComplete(CompletableFuture.java:510)
	at java.base/java.util.concurrent.CompletableFuture.complete(CompletableFuture.java:2179)
	at org.apache.flink.runtime.rpc.pekko.PekkoInvocationHandler.lambda$invokeRpc$1(PekkoInvocationHandler.java:268)
	at java.base/java.util.concurrent.CompletableFuture.uniWhenComplete(CompletableFuture.java:863)
	at java.base/java.util.concurrent.CompletableFuture$UniWhenComplete.tryFire(CompletableFuture.java:841)
	at java.base/java.util.concurrent.CompletableF

In [ ]:
env.close()